In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from data import CIFAR10
from ff import FF

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [3]:
batch_size = 32

In [4]:
trainset = torchvision.datasets.CIFAR10(root='./datasets', train=True,
                                        download=True, transform=transform)

Files already downloaded and verified


In [5]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [6]:
testset = torchvision.datasets.CIFAR10(root='./datasets', train=False,
                                       download=True, transform=transform)

Files already downloaded and verified


In [7]:
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

# Inference

In [8]:
config = {
    "in_dims": [3468, 3468, 3468],
    "out_dims": [3468, 3468, 3468],
    "epochs": 25,
    "threshold": 2
}
num_layers = 3

In [9]:
model = FF(num_layers, config, torch.optim.Adam)

In [10]:
model.train()

FF()

In [11]:
x = iter(trainloader)

In [12]:
x, y = next(x)

In [13]:
new_x, new_y = CIFAR10.overlay_y_on_x(x, y)

In [14]:
model(new_x)

torch.Size([4, 3072])
tensor([0.0001, 0.0001, 0.0001, 0.0001], grad_fn=<MeanBackward1>)
torch.Size([4, 3072])
tensor([0.0001, 0.0001, 0.0001, 0.0001], grad_fn=<MeanBackward1>)


tensor([0.0002, 0.0002, 0.0002, 0.0002], grad_fn=<AddBackward0>)

# Training

In [11]:
trainset = torchvision.datasets.CIFAR10(root='./datasets', train=True,
                                        download=True, transform=transform)

Files already downloaded and verified


In [12]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [ ]:
model.train()
for i, (x, y) in enumerate(trainloader):
    
    x_pos, _ =  CIFAR10.overlay_y_on_x(x, y)
    rnd = torch.randperm(x.size(0))
    x_neg, _ = CIFAR10.overlay_y_on_x(x, y[rnd])
    losses = model(x_pos, x_neg)
    print(losses)

0.7862356456120808
0.7092948150634765
0.701378071308136
0.6984734988212585
0.6936511484781901
0.6910433030128479
0.6941611536343893
0.6899163309733073
0.6888981310526531
0.6929790989557901
0.6811924934387207
0.6801064546902976
0.6773826670646667
0.6730228487650555
0.6700795896848043
0.6708218423525492
0.6707312099138895
0.6635757700602214
0.6626875178019206
0.6598532199859619
0.6933364168802897
0.6710074774424234
0.6535877641042074
